# Spark API Exercises

In [222]:
import pandas as pd
import numpy as np
import pyspark 
import pydataset

from pyspark.sql.functions import sum, mean, concat, lit, regexp_extract, regexp_replace, when, month, year, quarter, asc, desc, col,expr, count


In [2]:
# set up spark invironment
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

### 1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be language
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe


In [116]:
# create dictionary
Dict = {'language': ['python', 'php', 'C', 'C++', 'java', 'SQL']}

# turn dictionary into pandas dataframe
fav_languages = pd.DataFrame(Dict)

In [117]:
# turn pandas dataframe into spark dataframe
df = spark.createDataFrame(fav_languages)

In [118]:
# look at shape
print((df.count(), len(df.columns)))

(6, 1)


In [119]:
df.show(5)

+--------+
|language|
+--------+
|  python|
|     php|
|       C|
|     C++|
|    java|
+--------+
only showing top 5 rows



### 2. Load the mpg dataset as a spark dataframe.

    a. Create 1 column of output that contains a message like the one below:

    `The 1999 audi a4 has a 4 cylinder engine.`
    
    For each vehicle.
    
    b. Transform the trans column so that it only contains either manual or auto.

In [181]:
# load mpg data
mpg = pydataset.data('mpg')

# turn into spark dataframe
df = spark.createDataFrame(mpg)

In [182]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [183]:
df.select(concat(lit('The '), 'year', lit(' '), 'manufacturer', 
                 lit(' '), 'model', lit(' has a '), 'cyl', lit(' cylinder engine.')).alias('description'))

DataFrame[description: string]

In [184]:
df = df.select('*',concat(lit('The '), 'year', lit(' '), 'manufacturer',
                          lit(' '), 'model', lit(' has a '), 'cyl', lit(' cylinder engine.')).alias('description'))

df.show(5, truncate = False)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+-----------------------------------------+
|manufacturer|model|displ|year|cyl|trans     |drv|cty|hwy|fl |class  |description                              |
+------------+-----+-----+----+---+----------+---+---+---+---+-------+-----------------------------------------+
|audi        |a4   |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.|
|audi        |a4   |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.8  |1999|6  |auto(l5)  |f  |16 |26 |p  |compact|The 1999 audi a4 has a 6 cylinder engine.|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------------

In [192]:
df.select(
    'trans',
    regexp_extract('trans', r'(\w+)',1).alias('trans2')
).show(4)

+----------+------+
|     trans|trans2|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
+----------+------+
only showing top 4 rows



In [197]:
new_trans = df.select(regexp_extract('trans', r'(\w+)',1).alias('trans'))
new_trans.show(5)

+------+
| trans|
+------+
|  auto|
|manual|
|manual|
|  auto|
|  auto|
+------+
only showing top 5 rows



In [205]:
new_df = df.select('manufacturer', 
          'model', 
          'displ', 
          'year', 
          'cyl', 
          regexp_extract('trans', r'(\w+)',1).alias('trans_new'), 
          'drv', 
          'cty', 
          'hwy', 
          'fl', 
          'class', 
          'description')
new_df.show(5)

+------------+-----+-----+----+---+---------+---+---+---+---+-------+--------------------+
|manufacturer|model|displ|year|cyl|trans_new|drv|cty|hwy| fl|  class|         description|
+------------+-----+-----+----+---+---------+---+---+---+---+-------+--------------------+
|        audi|   a4|  1.8|1999|  4|     auto|  f| 18| 29|  p|compact|The 1999 audi a4 ...|
|        audi|   a4|  1.8|1999|  4|   manual|  f| 21| 29|  p|compact|The 1999 audi a4 ...|
|        audi|   a4|  2.0|2008|  4|   manual|  f| 20| 31|  p|compact|The 2008 audi a4 ...|
|        audi|   a4|  2.0|2008|  4|     auto|  f| 21| 30|  p|compact|The 2008 audi a4 ...|
|        audi|   a4|  2.8|1999|  6|     auto|  f| 16| 26|  p|compact|The 1999 audi a4 ...|
+------------+-----+-----+----+---+---------+---+---+---+---+-------+--------------------+
only showing top 5 rows



### 3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [36]:
from pyspark.sql.functions import sum, mean, concat, lit, regexp_extract, regexp_replace, when

In [267]:
tips = pydataset.data('tips')

df = spark.createDataFrame(tips)
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [270]:
# What percentage of observations are smokers?
(
    df.select('*', (when(df.smoker == 'Yes', 1).otherwise(0)).alias('is_smoker'))
    .agg(round(mean('is_smoker'), 2))
).show(5)

+------------------------+
|round(avg(is_smoker), 2)|
+------------------------+
|                    0.38|
+------------------------+



In [271]:
df.groupBy("smoker").count().withColumn(
    "percent",
    concat(round((col("count") / df.count() * 100), 0).cast("int"), lit("%")),
).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|    62%|
|   Yes|   93|    38%|
+------+-----+-------+



In [48]:
# Create a column that contains the tip percentage
df = df.select('*', round((df.tip / df.total_bill) *100, 2).alias('tip_percent'))

df.show(5)

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percent|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|       5.94|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|      16.05|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|      16.66|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|      13.98|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|      14.68|
+----------+----+------+------+---+------+----+-----------+
only showing top 5 rows



In [53]:
# Calculate the average tip percentage for each combination of sex and smoker.

df.groupBy('sex', 'smoker').agg(mean('tip_percent')).show(5)

+------+------+------------------+
|   sex|smoker|  avg(tip_percent)|
+------+------+------------------+
|  Male|    No| 16.06659793814433|
|  Male|   Yes|15.276666666666667|
|Female|    No| 15.69111111111111|
|Female|   Yes|18.214545454545455|
+------+------+------------------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

   - Convert the temperatures to fahrenheit.
   - Which month has the most rain, on average?
   - Which year was the windiest?
   - What is the most frequent type of weather in January?
   - What is the average high and low temperature on sunny days in July in 2013 and 2014?
   - What percentage of days were rainy in q3 of 2015?
   - For each year, find what percentage of days it rained (had non-zero precipitation).

In [208]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [209]:
# Convert the temperatures to fahrenheit

weather = weather.select('*', round(weather.temp_max * (9/5) + 32, 1).alias('temp_max(F)'))

weather = weather.select('*', round(weather.temp_min * (9/5) + 32, 1).alias('temp_min(F)'))

In [210]:
weather.show(5)

print(weather.count(), "rows", len(weather.columns), "columns")

+----------+-------------+--------+--------+----+-------+-----------+-----------+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max(F)|temp_min(F)|
+----------+-------------+--------+--------+----+-------+-----------+-----------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|       55.0|       41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|       51.1|       37.0|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|       53.1|       45.0|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|       54.0|       42.1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|       48.0|       37.0|
+----------+-------------+--------+--------+----+-------+-----------+-----------+
only showing top 5 rows

1461 rows 8 columns


In [211]:
# Which month has the most rain, on average?

# create datafame with the aggrigate rain totals for each month

rain = (
    weather.withColumn('month', month('date'))
    .groupby('month')
    .agg(sum('precipitation').alias('total_rain'))
    .sort('month')
    )

In [128]:
rain.show()

+-----+------------------+
|month|        total_rain|
+-----+------------------+
|    1|465.99999999999994|
|    2|             422.0|
|    3|             606.2|
|    4|             375.4|
|    5|             207.5|
|    6|             132.9|
|    7|              48.2|
|    8|             163.7|
|    9|235.49999999999997|
|   10|             503.4|
|   11|             642.5|
|   12| 622.7000000000002|
+-----+------------------+



In [129]:
rain.agg({'total_rain':'max'}).show()

+---------------+
|max(total_rain)|
+---------------+
|          642.5|
+---------------+



In [130]:
# month with the most rain
# sort by total_rain in descending and show top result

rain.sort(desc('total_rain')).show(1)

+-----+----------+
|month|total_rain|
+-----+----------+
|   11|     642.5|
+-----+----------+
only showing top 1 row



In [131]:
# month with the least rain
# sort by total_rain in ascending and show top result

rain.sort('total_rain').show(1)

+-----+----------+
|month|total_rain|
+-----+----------+
|    7|      48.2|
+-----+----------+
only showing top 1 row



In [132]:
# Which year was the windiest?
wind = (
    weather.withColumn('year', year('date'))
    .groupby('year')
    .agg(sum('wind').alias('total_wind'))
    .sort('year')
    )
wind.show()

+----+------------------+
|year|        total_wind|
+----+------------------+
|2012|            1244.7|
|2013|1100.8000000000006|
|2014|1236.5000000000007|
|2015|1153.3000000000002|
+----+------------------+



In [150]:
wind.agg({'total_wind':'max'}).show()

+---------------+
|max(total_wind)|
+---------------+
|         1244.7|
+---------------+



In [153]:
wind.sort(desc('total_wind')).show(1)

+----+----------+
|year|total_wind|
+----+----------+
|2012|    1244.7|
+----+----------+
only showing top 1 row



In [160]:
# What is the most frequent type of weather in January? 
(
    weather.withColumn('month', month('date')).crosstab('month', 'weather')
    .sort('month_weather')
 
).show(1)

+-------------+-------+---+----+----+---+
|month_weather|drizzle|fog|rain|snow|sun|
+-------------+-------+---+----+----+---+
|            1|     10| 38|  35|   8| 33|
+-------------+-------+---+----+----+---+
only showing top 1 row



Most frequent weather in January: Fog

In [212]:
# What is the average high and low temperature on sunny days in July in 2013 and 2014

# create year and month columns for fitering
weather2 = weather.withColumn('year', year('date')).withColumn('month', month('date'))
weather2.show(5)

+----------+-------------+--------+--------+----+-------+-----------+-----------+----+-----+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max(F)|temp_min(F)|year|month|
+----------+-------------+--------+--------+----+-------+-----------+-----------+----+-----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|       55.0|       41.0|2012|    1|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|       51.1|       37.0|2012|    1|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|       53.1|       45.0|2012|    1|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|       54.0|       42.1|2012|    1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|       48.0|       37.0|2012|    1|
+----------+-------------+--------+--------+----+-------+-----------+-----------+----+-----+
only showing top 5 rows



In [213]:
(
    weather2.filter((weather2.year == 2013) | (weather2.year == 2014))
    .filter(weather2.month == 7)
    .filter(weather2.weather == 'sun')
    .agg(round(mean('temp_max(F)'),2).alias('avg_max_temp(F)'), 
         round(mean('temp_min(F)'),2).alias('avg_min_temp(F)'))
    
).show()

+---------------+---------------+
|avg_max_temp(F)|avg_min_temp(F)|
+---------------+---------------+
|          80.29|          57.51|
+---------------+---------------+



In [252]:
# What percentage of days were rainy in q3 of 2015?

(
    weather.filter(quarter('date') == 3)
    .filter(year('date') == 2015)
    .select(when(col('weather') == 'rain', 1).otherwise(0).alias('rain'))
    .agg(round(mean('rain'), 2).alias('avg_rainy'))
    

).show()

+---------+
|avg_rainy|
+---------+
|     0.02|
+---------+



2% of days in q3 of 2015 were rainy

In [266]:
# For each year, find what percentage of days it rained (had non-zero precipitation).

(
    weather.withColumn('year', year('date'))
    .select('*',(when(weather.precipitation != 0, 1).otherwise(0)).alias('did_rain'))
    .groupBy('year')
    .agg(round(mean('did_rain'),2).alias("percentage_of_rainy_days"))

).show()

+----+------------------------+
|year|percentage_of_rainy_days|
+----+------------------------+
|2015|                    0.39|
|2013|                    0.42|
|2014|                    0.41|
|2012|                    0.48|
+----+------------------------+

